In [34]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

sector_etfs = ['XLE', 'XLF', 'XLK', 'XLI', 'XLB', 'XLV', 'XLY', 'XLP', 'XLU', 'XLRE'] # Add XLC?
sector_twin_etfs = [
    'XLF', 'VFH',   # Financials
    'XLK', 'VGT',   # Tech
    'XLY', 'VCR',   # Consumer Discretionary
    'XLP', 'VDC',   # Consumer Staples
    'XLV', 'VHT',   # Healthcare
    'XLE', 'VDE',   # Energy
    'XLI', 'VIS',   # Industrials
    'XLB', 'VAW',   # Materials
    'XLU', 'VPU',   # Utilities
    'XLRE', 'VNQ',  # Real Estate
    'VOX',   # Comms
]
dividend_etfs = ['VYM', 'HDV', 'SCHD', 'DVY']
bond_etfs = ['SHY', 'VGSH', 'IEF', 'TLH', 'TLT', 'SPTL']
industry_etfs = ['KBE', 'XOP', 'IHF', 'SOXX', 'ITA', 'IBB']
international_etfs = ['EFA', 'VEA', 'IEMG', 'EEM', 'EWJ', 'EWU', 'EWG', 'FXI', 'EWZ']
broad_index_etfs = ['SPY', 'VOO', 'IVV', 'DIA', 'IWM', 'QQQ', 'RSP', 'QQEW']
thematic_etfs = ['ARKK', 'ARKW', 'ROBO', 'BOTZ', 'TAN', 'ICLN', 'LIT'] # Add BATT?

etf_categories = {
    'sector': sector_etfs,
    'sector_twins': sector_twin_etfs,
    'dividend': dividend_etfs,
    'bond': bond_etfs,
    'international': international_etfs,
    'broad_index': broad_index_etfs,
    'thematic': thematic_etfs,
}

flattened_etfs = [etf for etfs in etf_categories.values() for etf in etfs]

prices = yf.download(flattened_etfs, start = '2014-01-01', end = '2019-12-31')['Close']

for etf in prices.columns:
    data = prices[etf]

    if data.empty:
        print(f"{etf} does not contain data")

# first_frame = ['2010-01-01', '2020-12-31']
# second_frame = ['2015-01-01, 2020-12-31']
# third_frame = ['2018-01-01', '2023-12-31']

for category, etfs in etf_categories.items():
    desired_etfs = [etf for etf in etfs]

    category_prices = prices[desired_etfs]

    returns = category_prices.pct_change().dropna()

    correlation_matrix = returns.corr()
    correlation_matrix = correlation_matrix.rename_axis(None).rename_axis(None, axis = 1)
    correlation_matrix = correlation_matrix.stack().reset_index()
    correlation_matrix.columns = ['ETF', 'Pair', 'Correlation']

    correlation_matrix =  correlation_matrix[correlation_matrix['ETF'] != correlation_matrix['Pair']]
    correlation_matrix = correlation_matrix.sort_values(by = 'Correlation', ascending = False)
    correlation_matrix = correlation_matrix.drop_duplicates('Correlation')

    potential_pairs = correlation_matrix[correlation_matrix['Correlation'] > 0.8]

    print(f"Potential Correlated Pairs from {category}:")
    print(potential_pairs)

/var/folders/qd/xclxh05s0x9509nblm6z434w0000gp/T/ipykernel_45596/1052486875.py:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(flattened_etfs, start = '2014-01-01', end = '2019-12-31')['Close']
[*********************100%***********************]  55 of 55 completed


Potential Correlated Pairs from sector:
    ETF Pair  Correlation
26  XLK  XLY     0.847599
43  XLB  XLI     0.834225
13  XLF  XLI     0.803239
Potential Correlated Pairs from sector_twins:
      ETF Pair  Correlation
221   XLE  VDE     0.995401
373   VPU  XLU     0.993256
153   VDC  XLP     0.992817
21    VFH  XLF     0.992749
45    XLK  VGT     0.992254
89    XLY  VCR     0.991976
285   VIS  XLI     0.991683
177   XLV  VHT     0.988901
329   VAW  XLB     0.984735
397  XLRE  VNQ     0.931970
288   VIS  VAW     0.867210
108   VCR  VGT     0.853578
107   VCR  XLK     0.849242
267   XLI  VAW     0.848994
67    VGT  XLY     0.848930
86    XLY  XLK     0.847599
307   XLB  VIS     0.843717
266   XLI  XLB     0.834225
278   VIS  VCR     0.825232
34    VFH  VIS     0.823906
273   VIS  XLF     0.818598
97    XLY  VIS     0.809019
33    VFH  XLI     0.803381
252   XLI  XLF     0.803239
117   VCR  XLI     0.802500
Potential Correlated Pairs from dividend:
     ETF  Pair  Correlation
2    VYM  SC

In [35]:
# Get number of non-NaN entries (i.e., valid data points) per ETF
etf_data_counts = prices.notna().sum().sort_values()

# Optional: convert to DataFrame for cleaner display
etf_data_coverage = etf_data_counts.to_frame(name='Available Days')
etf_data_coverage.index.name = 'ETF'

# Print ETFs with the least data
print(etf_data_coverage.head(10))  # change number to see more

      Available Days
ETF                 
BOTZ             830
XLRE            1064
ARKK            1299
ARKW            1322
VAW             1509
VCR             1509
VDC             1509
VDE             1509
VEA             1509
VFH             1509


In [ ]:
for category, etfs in etf_categories.items():

        candidate_pairs = []

        desired_etfs = [etf for etf in etfs]

        category_prices = prices[desired_etfs]

        for i in range(len(desired_etfs)):
                for j in range(i + 1, len(desired_etfs)):
                        candidate_pairs.append([desired_etfs[i], desired_etfs[j]])

        cointegrated_pairs = []

        for etf1, etf2 in candidate_pairs:

                df = category_prices[[etf1, etf2]].dropna()

                s_etf1 = df[etf1]
                s_etf2 = df[etf2]

                score, pvalue, _ = coint(s_etf1, s_etf2)

                if pvalue < 0.05:
                        cointegrated_pairs.append((etf1, etf2, pvalue))
                        
        cointegrated_pairs_df = pd.DataFrame(cointegrated_pairs, columns = ['ETF', 'Pair', 'P-Value'])
        cointegrated_pairs_df = cointegrated_pairs_df.sort_values('P-Value').reset_index(drop=True)

        print(f"Cointegration Test Results for {category}:")
        print(cointegrated_pairs_df)

Cointegration Test Results for sector:
   ETF  Pair   P-Value
0  XLU  XLRE  0.001565
1  XLF   XLB  0.018776
2  XLF   XLI  0.029098
Cointegration Test Results for sector_twins:
   ETF  Pair   P-Value
0  VPU  XLRE  0.001233
1  XLU  XLRE  0.001565
2  VFH   VIS  0.018134
3  XLF   VIS  0.018728
4  XLF   XLB  0.018776
5  XLU   VNQ  0.019230
6  VPU   VNQ  0.029016
7  XLF   XLI  0.029098
8  VFH   XLI  0.029958
Cointegration Test Results for dividend:
    ETF  Pair   P-Value
0   VYM   DVY  0.001377
1  SCHD   DVY  0.014505
2   VYM  SCHD  0.025274
Cointegration Test Results for bond:
Empty DataFrame
Columns: [ETF, Pair, P-Value]
Index: []
Cointegration Test Results for international:
    ETF Pair  P-Value
0  IEMG  FXI  0.01601
1   EEM  FXI  0.02569
Cointegration Test Results for broad_index:
   ETF  Pair   P-Value
0  QQQ   RSP  0.003843
1  IVV   QQQ  0.012280
2  VOO   QQQ  0.012502
3  RSP  QQEW  0.015811
4  SPY   RSP  0.017028
5  VOO   RSP  0.018348
6  IVV   RSP  0.019185
7  SPY   QQQ  0.020424
8

In [41]:
window_size = 504
min_passes = 0.6
step = 30

for category, etfs in etf_categories.items():

    candidate_pairs = []
    rolling_cointegrated_pairs = []

    desired_etfs = [etf for etf in etfs]

    category_prices = prices[desired_etfs]

    for i in range(len(desired_etfs)):
        for j in range(i + 1, len(desired_etfs)):
                candidate_pairs.append([desired_etfs[i], desired_etfs[j]])

    for etf1, etf2 in candidate_pairs:
            
        df = category_prices[[etf1, etf2]].dropna()

        s_etf1 = df[etf1]
        s_etf2 = df[etf2]

        df = pd.concat([s_etf1, s_etf2], axis = 1)

        if len(df[etf1]) == 0:
            print(f"{etf1} does not have sufficient data")
            continue
        elif len(df[etf2]) == 0:
            print(f"{etf2} does not have sufficient data")
            continue
            
        series1 = df.iloc[:, 0]
        series2 = df.iloc[:, 1]

        cointegrated_windows = 0
        total_windows = 0

        for start in range(0, len(df) - window_size + 1, step):
            end = start + window_size

            window_s1 = series1.iloc[start:end]
            window_s2 = series2.iloc[start:end]
                
            score, pvalue, _ = coint(window_s1, window_s2)
            total_windows += 1
                
            if pvalue < 0.05:
                cointegrated_windows += 1

        if cointegrated_windows / total_windows >= min_passes:
            rolling_cointegrated_pairs.append({'ETF1': etf1, 'ETF2': etf2, 'Pass %': cointegrated_windows / total_windows})


    rolling_cointegrated_pairs_df = pd.DataFrame(rolling_cointegrated_pairs)

    if rolling_cointegrated_pairs_df.empty:
        print(f"{category} has no rolling cointegrated pairs.")
        continue
    else:
        rolling_cointegrated_pairs_df = rolling_cointegrated_pairs_df.sort_values('Pass %', ascending = False).reset_index(drop=True)
        print(f"Rolling Cointegration Test Results for {category}:")
        print(rolling_cointegrated_pairs_df)

Rolling Cointegration Test Results for sector:
  ETF1  ETF2  Pass %
0  XLU  XLRE     1.0
Rolling Cointegration Test Results for sector_twins:
  ETF1  ETF2  Pass %
0  XLU  XLRE     1.0
1  VPU  XLRE     1.0
Rolling Cointegration Test Results for dividend:
  ETF1  ETF2    Pass %
0  VYM  SCHD  0.909091
bond has no rolling cointegrated pairs.
international has no rolling cointegrated pairs.
Rolling Cointegration Test Results for broad_index:
  ETF1 ETF2    Pass %
0  SPY  VOO  0.636364
1  VOO  IVV  0.636364
Rolling Cointegration Test Results for thematic:
   ETF1  ETF2  Pass %
0  ARKK  ARKW     1.0


In [44]:
prices = prices.dropna()

def zscore_calc(series):
    return (series - series.mean()) / series.std()

def adf_test(series):
    test_res = adfuller(series)
    return {'stat': test_res[0], 'p-value': test_res[1]}

def hedge_ratio_calc(series1, series2):
    x = sm.add_constant(series2)
    model = sm.OLS(series1, x).fit()

    return model.params.iloc[1]


for category, etfs in etf_categories.items():

    candidate_pairs = []
    rolling_cointegrated_pairs = []
    results = []

    desired_etfs = [etf for etf in etfs]

    category_prices = prices[desired_etfs]

    for i in range(len(desired_etfs)):
        for j in range(i + 1, len(desired_etfs)):
                candidate_pairs.append([desired_etfs[i], desired_etfs[j]])

    for etf1, etf2 in candidate_pairs:

        df = category_prices[[etf1, etf2]].dropna()

        s_etf1 = df[etf1]
        s_etf2 = df[etf2]

        hedge_ratio = hedge_ratio_calc(s_etf1, s_etf2)

        spread = s_etf1 - (hedge_ratio * s_etf2)

        zscore_spread = zscore_calc(spread)

        adf_res = adf_test(spread)

        results.append(
            {'ETF1': etf1,
            'ETF2': etf2,
            'adf_value': adf_res['stat'],
            'p-value': adf_res['p-value'],
            'mean': zscore_spread.mean(),
            'std': zscore_spread.std()}
        )

    results = pd.DataFrame(results)
    results = results.sort_values('p-value', ascending = True)
    results = results[results['p-value'] < 0.05]

    print(f"ADF Test Results for {category}:")
    print(results)

ADF Test Results for sector:
   ETF1  ETF2  adf_value   p-value          mean  std
44  XLU  XLRE  -4.431708  0.000261  8.560756e-18  1.0
11  XLF   XLB  -3.689733  0.004260  0.000000e+00  1.0
10  XLF   XLI  -3.536567  0.007096  0.000000e+00  1.0
9   XLF   XLK  -3.285976  0.015517  9.416831e-16  1.0
6   XLE   XLP  -3.016101  0.033418  2.071703e-15  1.0
29  XLI  XLRE  -2.988748  0.035959 -3.424302e-16  1.0
16  XLF  XLRE  -2.907800  0.044448  4.280378e-16  1.0
42  XLP   XLU  -2.871797  0.048727 -6.848605e-16  1.0
7   XLE   XLU  -2.864428  0.049644  1.369721e-15  1.0
ADF Test Results for sector_twins:
    ETF1  ETF2  adf_value   p-value          mean  std
204  VPU  XLRE  -4.495324  0.000201  1.712151e-17  1.0
201  XLU  XLRE  -4.431708  0.000261  8.560756e-18  1.0
31   VFH   VIS  -3.701383  0.004094  6.848605e-17  1.0
12   XLF   VIS  -3.690360  0.004251 -4.965238e-16  1.0
13   XLF   XLB  -3.689733  0.004260  0.000000e+00  1.0
202  XLU   VNQ  -3.681525  0.004381 -6.334959e-16  1.0
205  VPU   